In [268]:
import ee
import geemap.core as geemap
from datetime import date, timedelta

In [269]:
ee.Initialize(project="aquaticweeds7");
Map = geemap.Map();

In [270]:
print(ee.String('Hello from the Earth Engine servers!').getInfo())

Hello from the Earth Engine servers!


In [271]:
dateNow = date.today()
dateNow = dateNow - timedelta(days = 2);
lastDate = dateNow - timedelta(days = 30);
dateNow = dateNow.strftime("%Y-%m-%d")
lastDate = lastDate.strftime("%Y-%m-%d")

In [272]:
print(f"Today's date is: {dateNow}\n"
      f"The last image taken was: {lastDate}")

Today's date is: 2024-10-02
The last image taken was: 2024-09-02


In [275]:
image = ee.Image(ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA')
  .filterDate('2017-01-01', '2017-12-31')
  .filterBounds(ee.Geometry.Point(27.8569, -25.7381))
  .sort('CLOUD_COVER')
  .first())

# Calculate NDVI to highlight green vegetation (NDVI = (NIR - Red) / (NIR + Red))
ndvi = image.normalizedDifference(['B5', 'B4'])  # NIR = B5, Red = B4 for Landsat 8

# Apply a threshold to extract water hyacinth (NDVI > 0.3 is often vegetation)
hyacinth_mask = ndvi.gt(0.3)

# Optional: Apply morphology operations (erosion/dilation) to clean the mask
hyacinth_cleaned = hyacinth_mask.focal_max(1)  # Simple dilation

# Reduce to vectors with bestEffort and increased maxPixels
hyacinth_geometry = hyacinth_cleaned.reduceToVectors(
    geometryType='polygon', 
    reducer=ee.Reducer.countEvery(), 
    scale=30,  # Resolution (30m for Landsat)
    maxPixels=1e8,  # Increase maxPixels limit
    bestEffort=True  # Use best effort to reduce pixel count
)

# Calculate the centroid
centroid = hyacinth_geometry.geometry().centroid()

# Calculate the area of the mat (in square kilometers)
area = hyacinth_geometry.geometry().area().divide(1e6)  # Area in km²

# Add layers to the map
Map.setCenter(27.8569, -25.7381, 13)
Map.addLayer(ndvi, {'min': 0, 'max': 1, 'palette': ['white', 'green']}, 'NDVI')
Map.addLayer(hyacinth_cleaned.updateMask(hyacinth_cleaned), {'palette': ['green']}, 'Hyacinth Mask')
Map.addLayer(ee.Image().paint(hyacinth_geometry, 0, 2), {}, 'Water Hyacinth Mat')

area = area.getInfo()

# Print results
print(f"Mat Area: {area} km²")
print(f"Centroid: {centroid}")

Mat Area: ee.Number({
  "functionInvocationValue": {
    "functionName": "Number.divide",
    "arguments": {
      "left": {
        "functionInvocationValue": {
          "functionName": "Geometry.area",
          "arguments": {
            "geometry": {
              "functionInvocationValue": {
                "functionName": "Collection.geometry",
                "arguments": {
                  "collection": {
                    "functionInvocationValue": {
                      "functionName": "Image.reduceToVectors",
                      "arguments": {
                        "bestEffort": {
                          "constantValue": true
                        },
                        "geometryType": {
                          "constantValue": "polygon"
                        },
                        "image": {
                          "functionInvocationValue": {
                            "functionName": "Image.focal_max",
                            "arguments": {

In [254]:
Map

Map(bottom=4815348.0, center=[-25.728467541616347, 27.809357643127445], controls=(ZoomControl(options=['positi…

This is a test class